# Data Cleaning

raw HTML data
1. use regex to filter code and math tags
2. save filtered qids

ST pairs
1. match qids
2. exclude language sources, refer to https://stackexchange.com/sites#culturerecreation-traffic
3. further clean math ($) and code(```) using regex

## Set-Up

In [1]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 7.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import re

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd '/content/drive/MyDrive/WayangLLM'

/content/drive/MyDrive/WayangLLM


In [13]:
def printResult(raw_data, clean_data):
  raw_count = raw_data['train'].num_rows
  clean_count = clean_data['train'].num_rows

  print('Total Records')
  print('Raw Dataset: ', raw_count)
  print('Cleaned Dataset: ', clean_count)
  print('Total Data Removed: ', raw_count - clean_count)

## Filter StackExchange Raw HTML Data

In [ ]:
raw_data = load_dataset("HuggingFaceH4/stack-exchange-preferences")

Resolving data files:   0%|          | 0/758 [00:00<?, ?it/s]

In [ ]:
html_tags = [
    r'<code>(.{10,})<\/code>',
    r'<span class="math-container">(.{10,})<\/span>'
]

def filter_records(sample, tags):
    for tag in tags:
        answers_check = any(re.findall(tag, ans['text'], re.DOTALL) for ans in sample['answers'])
        question_check = re.findall(tag, sample['question'], re.DOTALL)

        # If the tag is found in either answers or question, exclude the sample
        if answers_check or question_check:
            return False
    else:
        return True

In [ ]:
raw_data_filtered = raw_data.filter(lambda sample: filter_records(sample, html_tags))

In [ ]:
printResult(raw_data, raw_data_filtered)

Total Records
Raw Dataset:  10807695
Cleaned Dataset:  1933905
Total Data Removed:  8873790


In [ ]:
filtered_qids = raw_data_filtered['train']['qid']

In [ ]:
with open('filtered_qids.txt', 'w') as f:
  for qid in filtered_qids:
    f.write(str(qid) + '\n')

## Filter StackExchange Pairs

In [6]:
SPLIT = 1

In [7]:
data_pairs = load_dataset(f"IndonesiaAI/stackexchange-paired-part-{SPLIT}-revision")

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6700458 [00:00<?, ? examples/s]

In [8]:
with open('filtered_qids.txt', 'r') as f:
  filtered_qids = f.readlines()

filtered_qids = set([int(qid.strip()) for qid in filtered_qids])

In [9]:
foreign_language_urls = [
    'https://esperanto.stackexchange.com/',
    'https://conlang.stackexchange.com/',
    'https://languagelearning.stackexchange.com/',
    'https://korean.stackexchange.com/',
    'https://ukrainian.stackexchange.com/',
    'https://russian.stackexchange.com/',
    'https://italian.stackexchange.com/',
    'https://portuguese.stackexchange.com/',
    'https://latin.stackexchange.com/',
    'https://chinese.stackexchange.com/',
    'https://french.stackexchange.com/',
    'https://spanish.stackexchange.com/',
    'https://rus.stackexchange.com/',
    'https://german.stackexchange.com/',
    'https://judaism.stackexchange.com/',
    'https://japanese.stackexchange.com/',
]

def clean_url(url):
  return url.lower().split('://')[1].split('.com')[0]

foreign_language_urls = [clean_url(url) for url in foreign_language_urls]
print(foreign_language_urls)

unwanted_expressions = [
    r"\$(.{10,})\$",
    r"\`(.{10,})\`"
]

['esperanto.stackexchange', 'conlang.stackexchange', 'languagelearning.stackexchange', 'korean.stackexchange', 'ukrainian.stackexchange', 'russian.stackexchange', 'italian.stackexchange', 'portuguese.stackexchange', 'latin.stackexchange', 'chinese.stackexchange', 'french.stackexchange', 'spanish.stackexchange', 'rus.stackexchange', 'german.stackexchange', 'judaism.stackexchange', 'japanese.stackexchange']


In [10]:
def filter_records(sample, qids, urls, expressions):
    if sample['qid'] in qids and clean_url(sample['metadata'][1]) not in urls:
        for exp in expressions:
            for field in ['question', 'response_j', 'response_k']:
                if re.findall(exp, sample[field], re.DOTALL):
                    return False
        return True
    return False

In [11]:
clean_data = data_pairs.filter(lambda sample: filter_records(sample, filtered_qids,
                                                             foreign_language_urls, unwanted_expressions))

Filter:   0%|          | 0/6700458 [00:00<?, ? examples/s]

In [14]:
printResult(data_pairs, clean_data)

Total Records
Raw Dataset:  6700458
Cleaned Dataset:  1056803
Total Data Removed:  5643655


In [15]:
clean_data.push_to_hub(f'IndonesiaAI/cleaned-data-split-{SPLIT}')

Pushing dataset shards to the dataset hub:   0%|          | 0/6 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/177 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/177 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/177 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/177 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/177 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/177 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/14 [00:00<?, ?it/s]